In [72]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib 
import re

from sklearn.base import TransformerMixin, BaseEstimator
class Transformer(TransformerMixin,BaseEstimator):
    def __init__(self):
        self.wt=[]
        self.ram=[]
        self.x_res=[]
        self.y_res=[]
        self.all_res=[]
        self.scrn_type=[]
        self.cpu_freq=[]
        self.cpu_type=[]
    def fit(self,dataset,y=None):
        self.dataset=dataset
        for i in list(dataset.columns):
            X=dataset[i]
            if i=='Weight':
                for j in X:
                    self.wt.append(float(j.split("kg")[0]))
        
            elif i=="Ram":
                for j in X:
                    self.ram.append(int(j.split("GB")[0]))
            
            
    
            elif i=="ScreenResolution":
                most_freq=X.value_counts().keys()[0]
                for j in X.values:
                    temp=re.search(r"(\d{3,4}x\d{3,4})",j).group()
                    temp2=j.split(temp)
                    temp=temp.split('x')
                    if temp2[0]=='':
                        self.scrn_type.append(most_freq)
                        self.x_res.append(int(temp[0]))
                        self.y_res.append(int(temp[1]))
                        self.all_res.append([most_freq,int(temp[0]),int(temp[1])])
                    else:
                        self.scrn_type.append(temp2[0])
                        self.x_res.append(int(temp[0]))
                        self.y_res.append(int(temp[1]))
                        self.all_res.append([temp2[0],int(temp[0]),int(temp[1])])
            
            elif i=="Cpu":
                for j in X.values:
                    temp=re.search(r'(\d{1}.\d{0,2}GHz)',j)
                    if temp:
                        temp0=re.split(r'(\d{1}.\d{0,2}GHz)',j)[0]
                        temp=temp.group().split('GHz')[0]
                        if re.search(r'(\d \d)',temp):
                            temp=temp.split(" ")[1]
                            self.cpu_freq.append(float(temp))
                        else:
                            self.cpu_freq.append(float(temp))
                
                        self.cpu_type.append(temp0)
                    else:
                        temp0=re.split(r'(\d{1}GHz)',j)[0]
                        temp=re.search(r'(\d{1}GHz)',j).group().split('GHz')[0]
                        self.cpu_freq.append(float(temp))
                        self.cpu_type.append(temp0)
            elif i not in ["Ram",'Cpu',"Weight","ScreenResolution","Price_euros","Inches"]:
                self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique()))) 
        return self
    
    def transform(self,dataset):
        self.dataset.drop("Ram",axis=1,inplace=True)
        self.dataset["Ram(GB)"]=self.ram
        self.dataset.drop("Cpu",axis=1,inplace=True)
        self.dataset["Cpu_freq(GHz)"]=self.cpu_freq
        self.dataset["Cpu_type"]=self.cpu_type
        self.dataset["X_Res"]=self.x_res
        self.dataset["Y_Res"]=self.y_res
        self.dataset["Screen_type"]=self.scrn_type
        self.dataset.drop("ScreenResolution",axis=1,inplace=True)
        self.dataset["Screen_type"]   = self.dataset["Screen_type"].replace(to_replace= self.dataset["Screen_type"] .unique(),value=range(len( self.dataset["Screen_type"] .unique())))
        self.dataset["Cpu_type"]   = self.dataset["Cpu_type"].replace(to_replace= self.dataset["Cpu_type"] .unique(),value=range(len( self.dataset["Cpu_type"] .unique())))
        self.dataset.drop("Weight",axis=1,inplace=True)
        self.dataset["Weight(Kg)"]=self.wt
        return self.dataset
       

    
            

                   
       

    
            


In [73]:
data=pd.read_csv("laptop_price.csv",encoding='latin-1')
data_test=data.drop("Price_euros",axis=1)
data.head(2)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94


In [74]:
features=data.drop("Price_euros",axis=1)
label=data["Price_euros"]

In [75]:
model=joblib.load('Finalmodel.pkl')

In [76]:
model

RandomForestRegressor(criterion='absolute_error', max_depth=9, max_features=6,
                      min_samples_leaf=2, min_samples_split=4, n_estimators=42)

In [77]:
transformer=Transformer()
transformed_testx=transformer.fit_transform(data_test)

/tmp/ipykernel_2103034/247390603.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))
/tmp/ipykernel_2103034/247390603.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))
/tmp/ipykernel_2103034/247390603.py:82: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_object

In [78]:
trainx,testx,trainy,testy=train_test_split(data_test,label,test_size=0.2)

In [79]:
trainy

244    1079.00
608    1299.00
111    1379.00
516     521.47
848    2277.00
        ...   
991     499.00
369    1750.00
262     722.00
97      485.00
69      949.00
Name: Price_euros, Length: 1042, dtype: float64

In [80]:
predict=model.predict(testx)

In [81]:
from scipy import stats
squared_errors= (testy-predict)**2
confidence = 0.95
prederror=np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,loc=squared_errors.mean(),scale=stats.sem(squared_errors))).mean()

In [82]:
max=testy.max()
min=testy.min()

In [83]:
percent=(1-prederror/(max-min))*100
print(f"Accuracy is {percent}%")

Accuracy is 95.3426453429017%


In [84]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import FunctionTransformer,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
def weight_transformer(X):
    wt=[]
    for i in X.values:
        wt.append([float(i[0].split("kg")[0])])
    print(wt)
    return wt

def Weight_name(function_transformer,feature_names_in):
    return ["(KG)"]

def ram_transformer(X):
    ram=[]
    print(X.values)
    for j in X.values:
        ram.append( [int(j[0].split("GB")[0])])
    return ram
def ram_name(function_transformer,feature_names_in):
    return ["(GB)"]
    

In [85]:
model2= joblib.load('final_model2.pkl')

In [86]:
weight_pipeline= make_pipeline(FunctionTransformer(weight_transformer,feature_names_out=Weight_name),StandardScaler())
ram_pipeline= make_pipeline(FunctionTransformer(ram_transformer,feature_names_out=ram_name),StandardScaler())
cat_pipeline=make_pipeline(OneHotEncoder(sparse_output=False),StandardScaler())
transfomer2=ColumnTransformer([
    ("Scaled",StandardScaler(),["Inches"]),
                               ("Transformed",cat_pipeline,['Company', 'Product', 'TypeName','ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'OpSys']),
                               ("Converted1",weight_pipeline,["Weight"]),
                               ("Converted",ram_pipeline,['Ram'])
                               ])

In [87]:
data

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [88]:
len(features.columns)

12

In [93]:
transformed_features_2=transfomer2.fit_transform(features)
trainx2,testx2,trainy2,testy2=train_test_split(transformed_features_2,label)
pred2=model2.predict(testx2)
# pred2=model2.predict(testx2)

[[1.37], [1.34], [1.86], [1.83], [1.37], [2.1], [2.04], [1.34], [1.3], [1.6], [1.86], [1.86], [1.83], [2.2], [0.92], [1.37], [2.2], [1.83], [2.2], [1.22], [0.98], [2.5], [1.86], [1.62], [1.91], [2.3], [1.35], [2.2], [1.88], [2.5], [1.89], [1.65], [2.71], [1.2], [1.35], [1.44], [2.1], [2.8], [1.86], [2.1], [2.0], [2.65], [2.3], [2.2], [2.77], [1.37], [2.2], [3.2], [2.3], [2.8], [0.69], [2.2], [1.86], [1.49], [2.4], [2.13], [1.91], [2.0], [2.43], [2.0], [2.8], [1.7], [1.7], [1.4], [1.86], [1.8], [1.86], [1.44], [1.9], [3.0], [1.252], [1.4], [2.2], [2.7], [2.1], [2.2], [2.2], [2.02], [2.2], [2.5], [1.88], [0.92], [1.63], [2.2], [2.1], [2.65], [1.83], [1.96], [2.2], [1.21], [2.45], [2.2], [1.49], [2.65], [1.25], [1.5], [2.2], [2.2], [2.0], [2.62], [1.91], [2.1], [2.2], [1.38], [1.252], [1.58], [1.85], [1.3], [2.2], [1.58], [1.5], [1.23], [1.37], [1.63], [1.21], [1.26], [1.63], [2.16], [2.0], [1.7], [2.1], [1.8], [2.2], [2.5], [2.1], [1.86], [1.63], [1.65], [2.8], [2.5], [2.36], [2.2], [1.6

/home/prashant/Study material/WORK_SPACES/Python/venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [94]:
from scipy import stats
squared_errors2= (testy2-pred2)**2
confidence = 0.95
prederror2=np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,loc=squared_errors.mean(),scale=stats.sem(squared_errors))).mean()
maxm=testy2.max()
minm= testy2.min()
percent2=(1-prederror/(maxm-minm))*100
print(f"Accuracy of model 2 is {percent2} %")


Accuracy of model 2 is95.31183108713641 %
